In [ ]:
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import logging

#  第一版-待將set改成id 不抓取搬遷 抓取關店

In [ ]:
# if not os.path.exists('./data'):
#     os.makedirs('./data')
# URL="http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/?baragain=1"

# res=requests.get(URL+"&p=1")
# soup=bs(res.text,"lxml")
# res.close()
# page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
# print(str(page)+"pages(all)")

# #初始set、頁
# WebsiteSet=set()
# pagebegin=1

# #如果已抓取，由中途開始繼續抓取 (抓取的資料set及頁面)
# if os.path.exists('./data/WebsiteSet.json'):
#     with open("./data/WebsiteSet.json") as f:
#         x=json.load(f)
#         WebsiteSet=set(x[dien]+"-"+dien for dien in x)
#         pagebegin=int(len(WebsiteSet)/15)

# if os.path.exists('./data/beginpage.txt'):
#     with open("./data/beginpage.txt") as p:
#         pagebegin=int(p.read())
        
# print("開始頁%s"%pagebegin)
        
# # headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}

# #會累積的變數>>i WebsiteSet
# for i in range(pagebegin,page+1):
#     res=requests.get(URL+"&p="+str(i))
#     if res.status_code==200:
#         soup=bs(res.text,'lxml')
#         dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")
#         dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家
#         dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"] for dien in dienlist 
#                   if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，並留下網址
#         WebsiteSet.update(dienlist)
#         if i%10==0:
#             print("-----\n已抓取第%s頁"%i)
#             print(len(WebsiteSet))
#             websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
#             print("實際剩餘%s，店名重複%s"%(len(websitedict),len(WebsiteSet)-len(websitedict)))
#             with open("./data/WebsiteSet.json","w") as f:
#                 json.dump(websitedict,f)
#             with open("./data/beginpage.txt","w") as p:
#                 p.write(str(i))
#     else:
#         print("status_code"+str(res.status_code))
#         print(len(WebsiteSet))
#         websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
#         with open("./data/WebsiteSet.json","w") as f:
#             json.dump(websitedict,f)
#         with open("./data/beginpage.txt","w") as p:
#             p.write(str(i))
            
# websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
# with open("./data/WebsiteSet.json","w") as f:
#     json.dump(websitedict,f)
# with open("./data/beginpage.txt","w") as p:
#     p.write(str(i))

In [ ]:
#將{店名：網址}格式的json改成{ID:網址}
# if os.path.exists('./data/WebsiteSet.json'):
#     with open("./data/WebsiteSet.json") as f:
#         x=json.load(f)
# y={x[dien].split("/")[-1]:x[dien] for dien in x}
# with open("./data/WebsiteSet.json","w") as f:
#     json.dump(y,f)

# 第二版 -存取資料改為 id:網址 (目前抓完約需5小)

In [ ]:
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import time
#確認檔案資料夾是否存在
if not os.path.exists('../data'):
    os.makedirs('../data')
    
#抓取網址(愛評網全台餐飲店)
URL="http://www.ipeen.com.tw/search/taiwan/000/0-0-0-0/%20/?bar=1"

#抓取總頁數
res=requests.get(URL+"&p=1")
soup=bs(res.text,"lxml")
res.close()
page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
print(str(page)+"pages(all)")

#初始網址set(不會重複)、抓取起始頁
WebsiteSet=set()
pagebegin=1

#如果曾抓取，由中途開始繼續抓取 (抓取的資料set讀入)
if os.path.exists('../data/WebsiteSet.json'):
    with open("../data/WebsiteSet.json") as f:
        x=json.load(f)
        WebsiteSet=set(x[dien] for dien in x)
        
#從過去抓取到的頁面開始，繼續抓取
if os.path.exists('../data/beginpage.txt'):
    with open("../data/beginpage.txt") as p:
        pagebegin=int(p.read())
        
print("開始頁%s"%pagebegin)
        
# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}


#抓取各店家的id:網址
#會累積的變數>>i WebsiteSet
for i in range(pagebegin,page+1):
    res=requests.get(URL+"&p="+str(i))
    if res.status_code==200:
        soup=bs(res.text,'lxml')
        dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")
        
#         dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家>改全抓
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"].split("-")[0] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，並留下網址>改不留店名
        WebsiteSet.update(dienlist)
        
        #每抓取30頁的網址，睡5秒以免被ban，並將資料已json格式儲存一次
        if i%30==0:
            print("-----\n抓取第%s頁"%i)
            print(len(WebsiteSet))
            time.sleep(5)

            websitedict={Website.split("/")[-1]:Website for Website in WebsiteSet}#改存{id:網址}格式
            print("實際剩餘%s，id重複%s"%(len(websitedict),len(WebsiteSet)-len(websitedict)))
            with open("../data/WebsiteSet.json","w") as f:
                json.dump(websitedict,f)
            with open("../data/beginpage.txt","w") as p:
                p.write(str(i))
    else:
        #無法進入頁面則顯示目前網址個數
        print("status_code"+str(res.status_code))
        print(len(WebsiteSet))
#         websitedict={Website.split("/")[-1]:Website for Website in WebsiteSet}#改存{id:網址}格式
#         with open("./data/WebsiteSet.json","w") as f:
#             json.dump(websitedict,f)
#         with open("./data/beginpage.txt","w") as p:
#             p.write(str(i))


#將所有資料抓取完畢後，在儲存最後一次
websitedict={Website.split("/")[-1]:Website for Website in WebsiteSet}#改存{id:網址}格式
with open("../data/WebsiteSet.json","w") as f:
    json.dump(websitedict,f)
with open("../data/beginpage.txt","w") as p:
    p.write(str(i))
print("已抓取完畢")

# try summary

In [ ]:
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import time
import pymongo
from pymongo import MongoClient
# from pymongo import UpdateOne#0125
import smtplib
from email.message import EmailMessage
b=time.time()

msg = EmailMessage()
msg.set_content(str(time.strftime("%Y/%m/%d %H:%M")))
msg['Subject'] = 'The IpeenWebsite Crawler has started to run'
msg['From'] = "ServerNet"
msg['To'] = 'andy.yuan@wowprime.com'
with smtplib.SMTP('192.168.2.1',25) as s:
    s.send_message(msg)




#確認檔案資料夾是否存在
if not os.path.exists('../data'):
    os.makedirs('../data')
    
#抓取網址(愛評網全台餐飲店)
URL="http://www.ipeen.com.tw/search/taiwan/000/0-0-0-0/%20/?bar=1"


#抓取總頁數
res=requests.get(URL+"&p=1")
soup=bs(res.text,"lxml")
res.close()
page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
print(str(page)+"pages(all)")

#初始網址set(不會重複)、抓取起始頁
WebsiteSet=set()
pagebegin=1


#從過去抓取到的頁面開始，繼續抓取
if os.path.exists('../data/beginpage.txt'):
    with open("../data/beginpage.txt") as p:
        pagebegin=int(p.read())

print("開始頁%s"%pagebegin)

# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}


#抓取各店家的id:網址
#會累積的變數>>i WebsiteSet
for i in range(pagebegin+1,page+1):
#         try:
#             res=requests.get(URL+"&p="+str(i))
#             if "RedisException" in res.text:
#                 print("requests太頻繁被卡，睡兩分鐘")
#                 time.sleep(120)
#                 res=requests.get(URL+"&p="+str(i))
#         except:
#             print("requests逾時，睡兩分鐘")
#             time.sleep(120)
#             res=requests.get(URL+"&p="+str(i))
    res = None
    while res is None:
        try:
            res=requests.get(URL+"&p="+str(i))
            while "RedisException" in res.text or "目前系統忙碌中" in res.text:
                print("requests太頻繁被卡，睡兩分鐘")
                time.sleep(120)
                res=requests.get(URL+"&p="+str(i))
        except:
            print("requests逾時，睡兩分鐘")
            time.sleep(120)


    if res.status_code==200:
        soup=bs(res.text,'lxml')
        try:
            dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")
        except:
            print("本頁已死")
            continue

#         dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家>改全抓
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"].split("-")[0] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，並留下網址>改不留店名
        WebsiteSet.update(dienlist)

        #每抓取15頁的網址，睡5秒以免被ban，並將資料已json格式儲存一次
        if i%15==0:
            print("-----\n已抓取第%s頁"%i)
            print(len(WebsiteSet))

            #-----------------------------------------------
#                 with open("../data/WebsiteSet.json","w") as f:
#                     json.dump(websitedict,f)
            #連Mongodb
            client = pymongo.MongoClient('localhost', 27017,username='j122085',password='850605')
            #連DB名
            db=client['rawData']
            #秀DB內的collection list
            collection = db.ipeenWebsite
            try:
#                     operations=[UpdateOne({"_id":Website.split("/")[-1]},
#                                           {'$set':{"website":Website}},
#                                           upsert=True) 
#                                 Website in WebsiteSet]
#                     collection.bulk_write(operations)
                # 如果資料內有相同_id會報錯，但所有非相同_id的資料都會insert
                collection.insert_many([{"_id":Website.split("/")[-1],"website":Website}for Website in WebsiteSet], ordered=False)
            except:
                print("some data has same _id")
            client.close()
            WebsiteSet=set()
            #-----------------------------------------------    
            with open("../data/beginpage.txt","w") as p:
                p.write(str(i))
            time.sleep(20)  
    else:
        #無法進入頁面則顯示目前網址個數
        print("status_code"+str(res.status_code))
        print(len(WebsiteSet))
        print("request出錯，睡30秒")
        time.sleep(30)
#         websitedict={Website.split("/")[-1]:Website for Website in WebsiteSet}#改存{id:網址}格式
#         with open("./data/WebsiteSet.json","w") as f:
#             json.dump(websitedict,f)
#         with open("./data/beginpage.txt","w") as p:
#             p.write(str(i))


#將所有資料抓取完畢後，在儲存最後一次        
#連Mongodb
client = pymongo.MongoClient('localhost', 27017,username='j122085',password='850605')
#連DB名
db=client['rawData']
#秀DB內的collection list
collection = db.ipeenWebsite
try:
    collection.insert_many([{"_id":Website.split("/")[-1],"website":Website}for Website in WebsiteSet], ordered=False)
except:
    print("some data has same _id")
NData=collection.count()
client.close()
with open("../data/beginpage.txt","w") as p:
    p.write(str(0))
print("已抓取完畢")
#兩週抓一次


e=time.time()

msg = EmailMessage()
msg.set_content("\n".join([str(time.strftime("%Y/%m/%d %H:%M")),
                           "use {} second".format(round(e-b)),
                           "we crawled {} datas".format(NData)]))
msg['Subject'] = 'The IpeenWebsit Crawler is finished'
msg['From'] = "ServerNet"
msg['To'] = 'andy.yuan@wowprime.com'
with smtplib.SMTP('192.168.2.1',25) as s:
    s.send_message(msg)

In [ ]:
import pymongo
import datetime
import json


#連Mongodb
client = pymongo.MongoClient('172.20.26.39', 27017,username='j122085',password='850605')
#連DB名
db=client['rawData']
#秀DB內的collection list
print(db.collection_names())
collection = db.ipeenWebsite
print(collection.count())
client.close()

# try to concurrent.futures 無法跑到一半存檔、會被BAN

In [ ]:
import concurrent.futures
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import logging
import time

In [ ]:
if not os.path.exists('../data'):
    os.makedirs('../data')

In [ ]:
def getlink(page,linkset,):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}
    res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/?baragain=1&p=1&p="+str(page),headers=headers)
    if res.status_code==200:
        soup=bs(res.text,'lxml')
        res.close()
        dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")#找尋店家資訊
        dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，留下網址
        linkset.update(dienlist)#將該頁的店list加入set中(順便去除重複)
#         if i%10==0:#每十頁存一次
#             print(len(linkset))
#             websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#             with open("./data/linkset.json","w") as f:
#                 json.dump(websitedict,f)
    else:#沒連線成功也存
        print("status_code="+str(res.status_code))
        res.close()
        print(len(linkset))
#         websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#         with open("./data/linkset.json","w") as f:
#             json.dump(websitedict,f)
#     websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
    return dienlist

In [ ]:
res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/?baragain=1&p=1&p=1")
soup=bs(res.text,"lxml")
res.close()
page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
print(str(page)+"pages")

WebsiteSet=set()
start = time.time()
i=0
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    jobInfo = [executor.submit(getlink, page, WebsiteSet) for page in range(1,page+1)]
    for future in concurrent.futures.as_completed(jobInfo):
        try:
            data = future.result()
            WebsiteSet.update(data)
        except Exception as e:
            print(e)
            print("zzz")
            pass
        
end = time.time()  
print("-----------------")
print(end-start)

websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
with open("../data/linkset.json","w") as f:
    json.dump(websitedict,f)

In [ ]:
with open("../data/linkSet.json") as f:
    x=json.load(f)

In [ ]:
len(x)

# try to change proxy 失敗

In [ ]:
import concurrent.futures
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import logging
import time
import re
if not os.path.exists('../data'):
    os.makedirs('../data')

In [ ]:
def getlink(page,linkset,proxylist):
    res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-0-0-0/?baragain=1&p="+str(page))
    if res.status_code==200:
        soup=bs(res.text,'lxml')
        res.close()
        dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")#找尋店家資訊
        dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，留下網址
#         linkset.update(dienlist)#將該頁的店list加入set中(順便去除重複)
#         if i%10==0:#每十頁存一次
#             print(len(linkset))
#             websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#             with open("./data/linkset.json","w") as f:
#                 json.dump(websitedict,f)
    else:#沒連線成功也存
        res.close()
        n=0
        res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-0-0-0/?baragain=1&p="+str(page),proxies=proxylist[n])
        while res.status_code!=200:
            n+=1
            res.close()
            res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-0-0-0/?baragain=1&p="+str(page),proxies=proxylist[n])
        dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")#找尋店家資訊
        dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，留下網址
#         websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#         with open("./data/linkset.json","w") as f:
#             json.dump(websitedict,f)
#     websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
    return dienlist

In [ ]:
url="https://free-proxy-list.net/"
res=requests.get(url)
soup=bs(res.text,"lxml")
proxylist=re.findall("([0-2]?[0-9]?[0-9]\.[0-2]?[0-9]?[0-9]\.[0-2]?[0-9]?[0-9]\.[0-2]?[0-9]?[0-9])(80)",soup.text)
proxylist=[{'http':'http://'+proxy[0]+':'+proxy[1]} for proxy in proxylist]
proxylist

In [ ]:
res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-0-0-0/?baragain=1&p=1")
soup=bs(res.text,"lxml")
res.close()
page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
print(str(page)+"pages")

WebsiteSet=set()
start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    jobInfo = [executor.submit(getlink, page, WebsiteSet,proxylist) for page in range(1,page+1)]
    for future in concurrent.futures.as_completed(jobInfo):
        try:
            data = future.result()
            WebsiteSet.update(data)
        except Exception as e:
            print(e)
            print("zzz")
            pass
        
end = time.time()  
print("-----------------")
print(end-start)

websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
with open("../data/linkset.json","w") as f:
    json.dump(websitedict,f)

In [ ]:
with open("../data/linkSet.json") as f:
    x=json.load(f)

# 變更proxy的兩種方法
import requests
s = requests.Session()
s.proxies = {"http": "http://61.233.25.166:80"}
r = s.get("http://www.google.com")
print(r.text)

import requests
r = requests.get("http://www.google.com", proxies={"http": "http://61.233.25.166:80"})
print(r.text)

# 變更user agent的方法
headers = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'youremail@domain.com'  # This is another valid field
}
response = requests.get(url, headers=headers)

# bs抓取某tag or tag群的方法
soup.select("tag > tag > tag")
如:soup.select("#search > article > hgroup > h2 > b")
    
soup.find_all('tag', attrs={'tag內的元素': '元素名稱'}) 
如:soup.find_all('a', attrs={'data-label': '店名'})